In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define the Generator class
class Discriminator(nn.Module):
    def __init__(self, vocab_size, hidden_dim):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(vocab_size * max_length, hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_dim, hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        # Flatten the input tensor
        x = x.view(x.size(0), -1)
        return self.model(x)


# Define the Discriminator class
class Discriminator(nn.Module):
    def __init__(self, vocab_size, hidden_dim):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(vocab_size, hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_dim, hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid()  # Output is a probability of being a real password
        )
    
    def forward(self, x):
        return self.model(x)

Using device: cuda


2. Password Dataset

In [ ]:
# Custom dataset class
class PasswordDataset(Dataset):
    def __init__(self, passwords, char2idx, max_length):
        self.passwords = passwords
        self.char2idx = char2idx
        self.max_length = max_length

    def __len__(self):
        return len(self.passwords)

    def __getitem__(self, idx):
        password = self.passwords[idx]
        padded_password = self.pad_sequence([self.char2idx[char] for char in password])
        return torch.tensor(padded_password, dtype=torch.long)
    
    def pad_sequence(self, seq):
        padded = [self.char2idx[' ']] * self.max_length
        padded[:len(seq)] = seq
        return padded

# Function to read passwords from a file
def read_passwords(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        passwords = [line.strip() for line in file.readlines()]
    return passwords

# Load passwords from the file
file_path = 'filtered_words_5_rockyou.txt'  # Replace with your file path
passwords = read_passwords(file_path)

# Create character mappings
char2idx = {char: idx for idx, char in enumerate(sorted(set(''.join(passwords) + ' ')))}
idx2char = {idx: char for char, idx in char2idx.items()}
max_length = max(len(password) for password in passwords)

# Create dataset and dataloader
dataset = PasswordDataset(passwords, char2idx, max_length)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)